Forward @ every step.

In [16]:
import torch
from spikingjelly.activation_based import neuron

net_s = neuron.IFNode(step_mode='s')
T = 4
N = 1
C = 3
H = 8
W = 8
x_seq = torch.rand([T, N, C, H, W])
print('x_seq.shape =', x_seq.shape)
print(x_seq)
y_seq = []
for t in range(T):
    x = x_seq[t]  # x.shape = [N, C, H, W]
    y = net_s(x)  # y.shape = [N, C, H, W]
    y_seq.append(y.unsqueeze(0))

y_seq = torch.cat(y_seq)
print('y_seq.shape =', y_seq.shape)
print(y_seq)
print(f"Parameters: step_mode={net_s.step_mode}, v_threshold={net_s.v_threshold}, v_reset={net_s.v_reset}")
# y_seq.shape = [T, N, C, H, W]

x_seq.shape = torch.Size([4, 1, 3, 8, 8])
tensor([[[[[1.3969e-01, 5.7241e-01, 1.3846e-01, 1.3894e-01, 6.3913e-01,
            1.5672e-01, 7.8673e-01, 1.7763e-01],
           [2.6963e-01, 9.1444e-02, 3.7533e-01, 2.1515e-01, 9.1215e-01,
            6.8916e-01, 8.0322e-01, 8.6394e-01],
           [4.4042e-01, 8.2779e-01, 1.6054e-01, 8.1819e-01, 3.4448e-01,
            9.9448e-01, 9.9542e-02, 8.3467e-01],
           [6.3582e-01, 6.0021e-01, 3.6065e-02, 3.7547e-02, 2.1857e-01,
            9.5829e-01, 3.1024e-01, 3.1122e-01],
           [7.9110e-02, 9.9801e-01, 5.5114e-01, 7.2318e-01, 7.3463e-01,
            7.1943e-01, 9.5675e-01, 7.7413e-01],
           [4.2640e-01, 1.7001e-01, 4.0806e-01, 7.2800e-01, 5.8326e-01,
            9.2726e-01, 4.0268e-01, 1.3796e-01],
           [8.1662e-01, 8.4596e-01, 2.1623e-02, 6.8984e-01, 2.4509e-02,
            6.9714e-01, 4.3843e-01, 4.1443e-01],
           [4.1302e-01, 5.8538e-01, 1.4243e-01, 9.0664e-02, 4.2742e-01,
            7.0951e-01, 5.6198e-01, 3.8

Forward @ multi-time step.

In [15]:
import torch
from spikingjelly.activation_based import neuron, functional

net_s = neuron.IFNode(step_mode='s')
T = 4
N = 1
C = 3
H = 8
W = 8
x_seq = torch.rand(T, N, C, H, W) 
print('x_seq.shape =', x_seq.shape)
# print(x_seq)
y_seq = functional.multi_step_forward(x_seq, net_s)   
print('y_seq.shape =', y_seq.shape)
# print(y_seq)

x_seq.shape = torch.Size([4, 1, 3, 8, 8])
y_seq.shape = torch.Size([4, 1, 3, 8, 8])


In [19]:
import torch
from spikingjelly.activation_based import neuron

net_m = neuron.IFNode(step_mode='m')
net_s = neuron.IFNode(step_mode='s')
T = 4
N = 1
C = 3
H = 8
W = 8
x_seq = torch.rand(T, N, C, H, W) 
print('x_seq.shape =', x_seq.shape)
# print(x_seq)
y_seq_m = net_m(x_seq)  # y_seq_m.shape = [T, N, C, H, W]
print('y_seq_m.shape =', y_seq_m.shape)
print('neuron shape: ', net_m.v.shape)
# print(y_seq_m)

x_seq.shape = torch.Size([4, 1, 3, 8, 8])
y_seq_m.shape = torch.Size([4, 1, 3, 8, 8])
neuron shape:  torch.Size([1, 3, 8, 8])


Propagate layer-by-layer.

In [ ]:
import torch
import torch.nn as nn
from spikingjelly.activation_based import neuron, functional, layer
T = 4
N = 2
C = 8
x_seq = torch.rand([T, N, C]) * 64.

net = nn.Sequential(
    layer.Linear(C, 4),
    neuron.IFNode(step_mode='m'),
    layer.Linear(4, 2),
    neuron.IFNode(step_mode='m')
)
print('net_structure', net)

with torch.no_grad():
    y_seq_layer_by_layer = x_seq
    for i in range(net.__len__()):
        layer_i = net[i]
        if isinstance(layer_i, neuron.IFNode):
            y_seq_layer_by_layer = functional.multi_step_forward(y_seq_layer_by_layer, layer_i)
        else:
            y_seq_layer_by_layer = layer_i(y_seq_layer_by_layer) 

net_structure Sequential(
  (0): Linear(in_features=8, out_features=4, bias=True)
  (1): IFNode(
    v_threshold=1.0, v_reset=0.0, detach_reset=False, step_mode=m, backend=torch
    (surrogate_function): Sigmoid(alpha=4.0, spiking=True)
  )
  (2): Linear(in_features=4, out_features=2, bias=True)
  (3): IFNode(
    v_threshold=1.0, v_reset=0.0, detach_reset=False, step_mode=m, backend=torch
    (surrogate_function): Sigmoid(alpha=4.0, spiking=True)
  )
)


AttributeError: 'Linear' object has no attribute 'v'